# Исследование базы данных StackOverflow

## Данные

Самостоятельная работа основана на базе данных StackOverflow — сервиса вопросов и ответов о программировании. 

Мы будем работать с версией базы, где хранятся данные о постах за 2008 год, но в таблицах есть  информация и о более поздних оценках, которые эти посты получили.

## Цель

Проект состоит из двух частей:

В первой части несколько задач в SQL-тренажёре, чтобы закрепить пройденный материал.

Вторая часть проекта — аналитическая.  


## Задания Часть 1


### Задание 1
Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».


In [ ]:
'''
WITH post_types as (SELECT *
FROM stackoverflow.post_types
WHERE type = 'Question'),

posts AS (SELECT *
     FROM stackoverflow.posts
     WHERE score > 300
     or favorites_count >= 100)

SELECT COUNT(*)
FROM post_types AS pt
JOIN posts AS p ON pt.id = p.post_type_id;

'''

### Задание 2
Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.



In [ ]:
'''
SELECT ROUND(AVG(question_cnt))    
FROM  (SELECT COUNT(*) AS question_cnt    
    FROM stackoverflow.posts AS p
LEFT JOIN stackoverflow.post_types AS pt on p.post_type_id = pt.id
    WHERE pt.type = 'Question' AND creation_date::date BETWEEN '2008-11-01' AND '2008-11-18'
 GROUP BY creation_date::date) AS Q;
    
'''

### Задание 3
Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.


In [ ]:
'''
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.users AS u
JOIN stackoverflow.badges AS b ON u.id=b.user_id
WHERE u.creation_date::date = b.creation_date::date; 
'''

### Задание 4

Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?



In [ ]:
'''
WITH Joel AS (SELECT DISTINCT p.id  AS id_
            FROM stackoverflow.posts AS p
            JOIN stackoverflow.users AS u ON p.user_id=u.id
            WHERE u.display_name = 'Joel Coehoorn'), 

dis AS (SELECT DISTINCT v.post_id as id1,
               COUNT(v.id) AS count_votes
        FROM stackoverflow.posts AS p
        JOIN stackoverflow.votes AS v ON p.id= v.post_id
        GROUP BY post_id)

SELECT COUNT(id1)
FROM Joel AS j
JOIN dis AS d ON j.id_= d.id1
WHERE count_votes > 0;
'''

### Задание 5
Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
'''
SELECT *,
ROW_NUMBER() OVER (ORDER BY id DESC) AS rank
FROM stackoverflow.vote_types
ORDER BY id;
'''

### Задание 6
Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
'''
SELECT v.user_id AS user_id,
       COUNT(vt.name) AS count_close
FROM stackoverflow.votes AS v
JOIN stackoverflow.vote_types AS vt ON v.vote_type_id = vt.id
WHERE vt.name = 'Close'
GROUP BY user_id
ORDER BY count_close desc
LIMIT 10;
'''

### Задание 7
Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно. Отобразите несколько полей:
идентификатор пользователя;
число значков;
место в рейтинге — чем больше значков, тем выше рейтинг.
Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
'''
SELECT DISTINCT b.user_id AS user_id,
       COUNT(b.id) AS count_badges,
       DENSE_RANK() OVER (ORDER BY COUNT(b.id)DESC) AS rank_
FROM stackoverflow.badges AS b
WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY user_id
ORDER BY count_badges DESC
LIMIT 10;      
'''

### Задание 8
Сколько в среднем очков получает пост каждого пользователя?

Сформируйте таблицу из следующих полей:

- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
'''
SELECT title,
       user_id,
       score,
       ROUND(AVG(score) OVER (PARTITION BY user_id))
FROM stackoverflow.posts
WHERE title IS NOT NULL 
AND score != 0;
'''

### Задание 9
Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
'''
SELECT title
FROM stackoverflow.posts
WHERE user_id IN
                (SELECT u.id as id
                FROM stackoverflow.users AS u 
                JOIN stackoverflow.badges AS b ON b.user_id=u.id
                GROUP BY u.id
                HAVING COUNT(b.id) > 1000)
AND title IS NOT NULL;
'''

### Задание 10
Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:

- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.


In [ ]:
'''
SELECT id,
       views,
       CASE
           WHEN views >= 350 THEN 1
           WHEN views < 350 AND views >= 100 THEN 2
           ELSE 3
       END
FROM stackoverflow.users
WHERE location LIKE '%United States%'
AND views != 0;
'''

### Задание 11
Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. 

Выведите поля с идентификатором пользователя, группой и количеством просмотров. 

Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
'''
WITH us_users  AS (SELECT id AS user_id,
       views AS count_views ,
       CASE
           WHEN views >= 350 THEN 1
           WHEN views < 350 AND views >= 100 THEN 2
           ELSE 3
       END AS groups
FROM stackoverflow.users
WHERE location LIKE '%United States%'
AND views != 0)

SELECT  user_id,
       groups,
       count_views       
FROM (SELECT  user_id,
       groups,
       count_views,
       MAX(count_views) OVER (PARTITION BY groups) AS max_views
       FROM us_users) AS max_us
WHERE  count_views = max_views    
ORDER BY max_views DESC, user_id;
'''

### Задание 12
Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:

- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [ ]:
'''
SELECT days,
       count_id,
       SUM(count_id) OVER (ORDER BY days)
FROM (       
SELECT EXTRACT(DAY FROM creation_date) AS days,
       COUNT(id) AS count_id
FROM stackoverflow.users
WHERE creation_date :: date BETWEEN '2008-11-01' AND '2008-11-30'
GROUP BY days
     ) AS Q ;
'''

### Задание 13
Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста.

Отобразите:

- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [ ]:
'''
WITH first_post AS
(SELECT DISTINCT user_id as user_id,
       FIRST_VALUE(creation_date) OVER (PARTITION BY user_id ORDER BY creation_date) AS first_post
FROM stackoverflow.posts
WHERE creation_date IS NOT NULL)

SELECT f.user_id, 
       f.first_post - creation_date as interval_
FROM stackoverflow.users AS u 
JOIN first_post AS f ON u.id = f.user_id;
'''



## Задания Часть 2 

### Задание 1

Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. 

Результат отсортируйте по убыванию общего количества просмотров.


In [ ]:
'''
SELECT DATE_TRUNC('month', creation_date)::date,
       SUM(views_count) AS sum_views
FROM stackoverflow.posts
GROUP BY DATE_TRUNC('month', creation_date)::date
ORDER BY sum_views DESC;
'''

### Задание 2

Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. 

Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. 

Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
'''
SELECT u.display_name AS users_name,
       COUNT(DISTINCT p.user_id)
FROM stackoverflow.users u 
JOIN stackoverflow.posts p ON p.user_id = u.id
JOIN stackoverflow.post_types pt ON pt.id = p.post_type_id
WHERE p.creation_date::date BETWEEN U.creation_date::date AND U.creation_date::date + INTERVAL '1 month'
AND pt.type = 'Answer'
GROUP BY u.display_name
HAVING COUNT(DISTINCT p.id) > 100       
ORDER BY users_name; 
'''

### Задание 3
Выведите количество постов за 2008 год по месяцам. 

Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года.

Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
'''
SELECT DATE_TRUNC('month', p.creation_date)::date as date_,
       COUNT(p.id) AS count_posts
FROM stackoverflow.posts p
WHERE  p.user_id IN (SELECT u.id
                    FROM stackoverflow.posts p
                    JOIN stackoverflow.users u ON u.id = p.user_id
                    WHERE DATE_TRUNC('month', u.creation_date)::date BETWEEN '2008-09-01' AND '2008-09-30'
                    AND DATE_TRUNC('month', p.creation_date)::date BETWEEN '2008-12-01' AND '2008-12-31'
                     ) 
GROUP BY DATE_TRUNC('month', p.creation_date)::date
ORDER BY DATE_TRUNC('month', p.creation_date)::date DESC;

'''

### Задание 4
Используя данные о постах, выведите несколько полей:

- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумму просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
'''
SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts     
ORDER BY user_id, creation_date;
'''

### Задание 5
Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? 

Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [ ]:
'''
SELECT ROUND(AVG(count_active_day))
FROM (SELECT user_id as user_id,
       COUNT(DISTINCT creation_date::date) AS count_active_day
FROM stackoverflow.posts 
WHERE creation_date BETWEEN '2008-12-01' AND '2008-12-07'
GROUP BY user_id)  AS active_day; 

'''

### Задание 6
На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? 
Отобразите таблицу со следующими полями:
- номер месяца;
- количество постов за месяц;
- процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.


Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.


In [ ]:
'''
SELECT month_number,
       count_post,
       ROUND((count_post::numeric / LAG(count_post) OVER (ORDER BY month_number)- 1)*100, 2) AS change_percent
FROM (SELECT extract(MONTH FROM creation_date) AS month_number,
       COUNT(id) as count_post
FROM stackoverflow.posts   
WHERE creation_date BETWEEN '01-09-2008' AND '31-12-2008'
GROUP BY month_number) AS Q;
'''

### Задание 7

Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
'''
SELECT DISTINCT
       EXTRACT(WEEK FROM creation_date) AS week_number, 
       MAX(creation_date) OVER (PARTITION BY EXTRACT(WEEK FROM creation_date)) AS last_post       
FROM stackoverflow.posts
WHERE user_id = (SELECT user_id                         
                    FROM stackoverflow.posts
                    GROUP BY user_id
                    ORDER BY COUNT(id) DESC
                    LIMIT 1)                    
AND creation_date::date BETWEEN '2008-10-01' AND '2008-10-31';
'''   